In [ ]:
import pickle
import pandas as pd
from pathlib import Path
import requests

In [ ]:
from moduleMeteo import geocode_postal_code, fetch_daily_forecast, _check_conditions, send_sms

In [ ]:
pkl_path = Path("cp_magasins.pkl")

In [ ]:
if not pkl_path.exists():
    raise FileNotFoundError(f"Fichier introuvable: {pkl_path.resolve()}")

with pkl_path.open("rb") as f:
    raw_obj = pickle.load(f)

if isinstance(raw_obj, pd.DataFrame):
    df_magasins = raw_obj.copy()
elif isinstance(raw_obj, dict):
    df_magasins = pd.DataFrame([raw_obj])
elif isinstance(raw_obj, (list, tuple)):
    dict_items = [x for x in raw_obj if isinstance(x, dict)]
    if dict_items:
        df_magasins = pd.DataFrame(dict_items)
    else:
        df_magasins = pd.DataFrame(raw_obj)
else:
    raise ValueError(f"Type pickle non supporté: {type(raw_obj).__name__}")

for col in ["magasin", "codes_postaux", "conditions_meteo", "delai_jours", "derniere_campagne"]:
    if col not in df_magasins.columns:
        df_magasins[col] = None

if "derniere_campagne" in df_magasins.columns:
    df_magasins["derniere_campagne"] = pd.to_datetime(df_magasins["derniere_campagne"], errors="coerce")

df_magasins.head()

,magasin,codes_postaux,conditions_meteo,delai_jours,derniere_campagne
0,Magasin Lille,"[59000, 59100, 59200]","{'temp_min': 7, 'temp_max': 10, 'precipitation...",5,2025-10-01
1,Magasin Paris,"[75001, 75002, 75003]","{'temp_min': 9, 'temp_max': 13, 'precipitation...",5,2025-09-28
2,Magasin Marseille,"[13001, 13002, 13003]","{'temp_min': 14, 'temp_max': 22, 'precipitatio...",5,2025-10-10
3,Magasin Lyon,"[69001, 69002, 69003]","{'temp_min': 9, 'temp_max': 14, 'precipitation...",5,2025-09-25
4,Magasin Bordeaux,"[33000, 33100, 33200]","{'temp_min': 10, 'temp_max': 14, 'precipitatio...",5,2025-10-05


In [ ]:
if "codes_postaux" not in df_magasins.columns:
    raise KeyError("La colonne 'codes_postaux' est absente de df_magasins")

def _to_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    if isinstance(x, (tuple, set)):
        return list(x)
    if isinstance(x, str):
        sx = x.strip()
        if sx.startswith("[") and sx.endswith("]"):
            try:
                return [s.strip().strip("'\"") for s in sx[1:-1].split(",") if s.strip()]
            except Exception:
                return [sx]
        return [sx]
    return [x]

_df = df_magasins.copy()
_df["codes_postaux"] = _df["codes_postaux"].apply(_to_list)

codes_exploded = _df.explode("codes_postaux", ignore_index=True).rename(columns={"codes_postaux": "code_postal"})

codes_exploded.head()

,magasin,code_postal,conditions_meteo,delai_jours,derniere_campagne
0,Magasin Lille,59000,"{'temp_min': 7, 'temp_max': 10, 'precipitation...",5,2025-10-01
1,Magasin Lille,59100,"{'temp_min': 7, 'temp_max': 10, 'precipitation...",5,2025-10-01
2,Magasin Lille,59200,"{'temp_min': 7, 'temp_max': 10, 'precipitation...",5,2025-10-01
3,Magasin Paris,75001,"{'temp_min': 9, 'temp_max': 13, 'precipitation...",5,2025-09-28
4,Magasin Paris,75002,"{'temp_min': 9, 'temp_max': 13, 'precipitation...",5,2025-09-28


In [ ]:
records = []
for _, row in codes_exploded.iterrows():
    cp = str(row["code_postal"]) if row["code_postal"] is not None else None
    if not cp:
        continue

    coords = geocode_postal_code(cp)
    if not coords:
        continue

    lat, lon = coords
    daily = fetch_daily_forecast(lat, lon)
    if not daily:
        continue

    times = daily.get("time", [])
    tmin = daily.get("temperature_2m_min", [])
    tmax = daily.get("temperature_2m_max", [])
    rain = daily.get("precipitation_sum", [])

    for i in range(min(len(times), 5)):
        records.append({
            "code_postal": cp,
            "date": times[i],
            "temp_min": tmin[i] if i < len(tmin) else None,
            "temp_max": tmax[i] if i < len(tmax) else None,
            "precipitation_sum": rain[i] if i < len(rain) else None,
        })

meteo_5j = pd.DataFrame.from_records(records)
meteo_5j.head()

,code_postal,date,temp_min,temp_max,precipitation_sum
0,59000,2025-10-20,12.8,15.6,5.5
1,59000,2025-10-21,11.8,14.6,17.9
2,59000,2025-10-22,11.0,14.6,3.0
3,59000,2025-10-23,10.0,11.6,50.8
4,59000,2025-10-24,6.8,10.4,3.9


In [7]:
if "code_postal" not in codes_exploded.columns:
    raise KeyError("'codes_exploded' doit contenir la colonne 'code_postal'")
if "code_postal" not in meteo_5j.columns:
    raise KeyError("'meteo_5j' doit contenir la colonne 'code_postal'")

m5 = meteo_5j.sort_values(["code_postal", "date"]).copy()
m5["day_rank"] = m5.groupby("code_postal").cumcount() + 1
m5_day5 = m5[m5["day_rank"] == 5][["code_postal", "date", "temp_min", "temp_max", "precipitation_sum"]]

bdd_finale = codes_exploded.merge(
    m5_day5,
    on="code_postal",
    how="left",
)

grouped_par_magasin = (
    bdd_finale.groupby(["magasin", "date"], dropna=False)
    .agg({
        "code_postal": "first",
        "temp_min": "mean",
        "temp_max": "mean",
        "precipitation_sum": "mean",
        "delai_jours": "first",
        "derniere_campagne": "first",
    })
    .reset_index()
)

grouped_par_magasin = grouped_par_magasin.merge(
    df_magasins[["magasin", "conditions_meteo"]].drop_duplicates("magasin"),
    on="magasin",
    how="left",
)

bdd_finale.head(), grouped_par_magasin.head()


(         magasin code_postal  \
 0  Magasin Lille       59000   
 1  Magasin Lille       59100   
 2  Magasin Lille       59200   
 3  Magasin Paris       75001   
 4  Magasin Paris       75002   
 
                                     conditions_meteo  delai_jours  \
 0  {'temp_min': 7, 'temp_max': 10, 'precipitation...            5   
 1  {'temp_min': 7, 'temp_max': 10, 'precipitation...            5   
 2  {'temp_min': 7, 'temp_max': 10, 'precipitation...            5   
 3  {'temp_min': 9, 'temp_max': 13, 'precipitation...            5   
 4  {'temp_min': 9, 'temp_max': 13, 'precipitation...            5   
 
   derniere_campagne        date  temp_min  temp_max  precipitation_sum  
 0        2025-10-01  2025-10-24       6.8      10.4                3.9  
 1        2025-10-01  2025-10-24       6.6      10.4                4.2  
 2        2025-10-01  2025-10-24       6.6      10.4                4.2  
 3        2025-09-28  2025-10-24       8.6      13.2                0.9  
 4      

In [8]:
if "code_postal" not in codes_exploded.columns:
    raise KeyError("'codes_exploded' doit contenir la colonne 'code_postal'")
if "code_postal" not in meteo_5j.columns:
    raise KeyError("'meteo_5j' doit contenir la colonne 'code_postal'")

m5 = meteo_5j.sort_values(["code_postal", "date"]).copy()
m5["day_rank"] = m5.groupby("code_postal").cumcount() + 1

# Créer un mapping pour récupérer le délai spécifique à chaque magasin
delai_mapping = codes_exploded[["magasin", "code_postal", "delai_jours"]].drop_duplicates()

# Fusionner avec les données météo pour avoir le délai pour chaque code postal
m5_with_delai = m5.merge(delai_mapping[["code_postal", "delai_jours"]], on="code_postal", how="left")

# Sélectionner le jour correspondant au délai spécifique de chaque magasin
m5_target_day = m5_with_delai[m5_with_delai["day_rank"] == m5_with_delai["delai_jours"]][["code_postal", "date", "temp_min", "temp_max", "precipitation_sum"]]

bdd_finale = codes_exploded.merge(
    m5_target_day,
    on="code_postal",
    how="left",
)

grouped_par_magasin = (
    bdd_finale.groupby(["magasin", "date"], dropna=False)
    .agg({
        "code_postal": "first",
        "temp_min": "mean",
        "temp_max": "mean",
        "precipitation_sum": "mean",
        "delai_jours": "first",
        "derniere_campagne": "first",
    })
    .reset_index()
)

grouped_par_magasin = grouped_par_magasin.merge(
    df_magasins[["magasin", "conditions_meteo"]].drop_duplicates("magasin"),
    on="magasin",
    how="left",
)

print("=== NOUVELLE LOGIQUE AVEC DELAI_JOURS SPÉCIFIQUE ===")
print("Chaque magasin utilise son propre délai pour la prédiction météo")
print()
print("Délais par magasin:")
for i, row in grouped_par_magasin.iterrows():
    print(f"{row['magasin']}: délai de {row['delai_jours']} jours")

bdd_finale.head(), grouped_par_magasin.head()


=== NOUVELLE LOGIQUE AVEC DELAI_JOURS SPÉCIFIQUE ===
Chaque magasin utilise son propre délai pour la prédiction météo

Délais par magasin:
Magasin Bordeaux: délai de 5 jours
Magasin Lille: délai de 5 jours
Magasin Lyon: délai de 5 jours
Magasin Marseille: délai de 5 jours
Magasin Nice: délai de 5 jours
Magasin Paris: délai de 5 jours


(         magasin code_postal  \
 0  Magasin Lille       59000   
 1  Magasin Lille       59100   
 2  Magasin Lille       59200   
 3  Magasin Paris       75001   
 4  Magasin Paris       75002   
 
                                     conditions_meteo  delai_jours  \
 0  {'temp_min': 7, 'temp_max': 10, 'precipitation...            5   
 1  {'temp_min': 7, 'temp_max': 10, 'precipitation...            5   
 2  {'temp_min': 7, 'temp_max': 10, 'precipitation...            5   
 3  {'temp_min': 9, 'temp_max': 13, 'precipitation...            5   
 4  {'temp_min': 9, 'temp_max': 13, 'precipitation...            5   
 
   derniere_campagne        date  temp_min  temp_max  precipitation_sum  
 0        2025-10-01  2025-10-24       6.8      10.4                3.9  
 1        2025-10-01  2025-10-24       6.6      10.4                4.2  
 2        2025-10-01  2025-10-24       6.6      10.4                4.2  
 3        2025-09-28  2025-10-24       8.6      13.2                0.9  
 4      

In [9]:
df_magasins_5j = df_magasins.copy()

# Utiliser la nouvelle logique avec delai_jours spécifique
m5 = meteo_5j.sort_values(["code_postal", "date"]).copy()
m5["day_rank"] = m5.groupby("code_postal").cumcount() + 1

# Créer un mapping pour récupérer le délai spécifique à chaque magasin
delai_mapping = codes_exploded[["magasin", "code_postal", "delai_jours"]].drop_duplicates()

# Fusionner avec les données météo pour avoir le délai pour chaque code postal
m5_with_delai = m5.merge(delai_mapping[["code_postal", "delai_jours"]], on="code_postal", how="left")

# Sélectionner le jour correspondant au délai spécifique de chaque magasin
m5_target_day = m5_with_delai[m5_with_delai["day_rank"] == m5_with_delai["delai_jours"]][["code_postal", "date", "temp_min", "temp_max", "precipitation_sum"]]

magasin_cp = codes_exploded[["magasin", "code_postal"]].drop_duplicates()
check_base = magasin_cp.merge(m5_target_day, on="code_postal", how="left")
check_base = check_base.merge(df_magasins_5j[["magasin", "conditions_meteo"]].drop_duplicates("magasin"), on="magasin", how="left")


In [10]:
check_base["conditions_ok_j5"] = check_base.apply(_check_conditions, axis=1)

conditions_par_magasin = (
    check_base.groupby("magasin", dropna=False)["conditions_ok_j5"].any().reset_index()
)

grouped_par_magasin = grouped_par_magasin.merge(conditions_par_magasin, on="magasin", how="left")

print("=== RÉSULTATS AVEC DELAI_JOURS SPÉCIFIQUE ===")
print("Chaque magasin utilise son délai personnalisé pour la prédiction météo")

check_base.head(), grouped_par_magasin.head()

=== RÉSULTATS AVEC DELAI_JOURS SPÉCIFIQUE ===
Chaque magasin utilise son délai personnalisé pour la prédiction météo


(         magasin code_postal        date  temp_min  temp_max  \
 0  Magasin Lille       59000  2025-10-24       6.8      10.4   
 1  Magasin Lille       59100  2025-10-24       6.6      10.4   
 2  Magasin Lille       59200  2025-10-24       6.6      10.4   
 3  Magasin Paris       75001  2025-10-24       8.6      13.2   
 4  Magasin Paris       75002  2025-10-24       8.6      13.2   
 
    precipitation_sum                                   conditions_meteo  \
 0                3.9  {'temp_min': 7, 'temp_max': 10, 'precipitation...   
 1                4.2  {'temp_min': 7, 'temp_max': 10, 'precipitation...   
 2                4.2  {'temp_min': 7, 'temp_max': 10, 'precipitation...   
 3                0.9  {'temp_min': 9, 'temp_max': 13, 'precipitation...   
 4                0.9  {'temp_min': 9, 'temp_max': 13, 'precipitation...   
 
    conditions_ok_j5  
 0             False  
 1             False  
 2             False  
 3              True  
 4              True  ,
           

In [11]:
grouped_par_magasin


,magasin,date,code_postal,temp_min,temp_max,precipitation_sum,delai_jours,derniere_campagne,conditions_meteo,conditions_ok_j5
0,Magasin Bordeaux,2025-10-24,33000,9.800000,14.300000,0.9,5,2025-10-05,"{'temp_min': 10, 'temp_max': 14, 'precipitatio...",True
1,Magasin Lille,2025-10-24,59000,6.666667,10.400000,4.1,5,2025-10-01,"{'temp_min': 7, 'temp_max': 10, 'precipitation...",False
2,Magasin Lyon,2025-10-24,69001,8.500000,14.400000,1.2,5,2025-09-25,"{'temp_min': 9, 'temp_max': 14, 'precipitation...",True
3,Magasin Marseille,2025-10-24,13001,13.266667,22.833333,0.0,5,2025-10-10,"{'temp_min': 14, 'temp_max': 22, 'precipitatio...",False
4,Magasin Nice,2025-10-24,06000,12.000000,21.900000,0.0,5,2025-10-14,"{'temp_min': 13, 'temp_max': 21, 'precipitatio...",True
5,Magasin Paris,2025-10-24,75001,8.600000,13.200000,0.9,5,2025-09-28,"{'temp_min': 9, 'temp_max': 13, 'precipitation...",True
